<a href="https://colab.research.google.com/github/Windflaw99/ssm-seoul-data-engineer/blob/master/02_python_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install selenum

In [2]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.hoDAS2s0oD/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.yUz60RAAOt/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.siQrjhl55l/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 http://de

In [3]:
from selenium import webdriver

In [8]:
from selenium.webdriver.common.by import By

In [15]:
chrome_options = webdriver.ChromeOptions()             # 웹 드라이버 옵션 지정
chrome_options.add_argument('--headless')              # 웹 브라우저 띄우지 않음
chrome_options.add_argument('--no-sandbox')            # 별도 웹페이지 작동 해제
chrome_options.add_argument('--disable-dev-shm-usage')

In [17]:
# browser = webdriver.Chrome('chromedriver', options=chrome_options)

browser = webdriver.Chrome(options=chrome_options)

In [27]:
browser.get("https://www.naver.com")
browser.title

'NAVER'

In [20]:
browser.get("https://www.google.com")
browser.title

'Google'

In [28]:
browser.find_element(By.CLASS_NAME, 'shortcut_list').text

'메일\n카페\n블로그\n쇼핑\n뉴스\n증권\n부동산\n지도\n웹툰\n바로가기 펼침'

In [57]:
browser.get("https://weather.naver.com")

In [58]:
word = browser.find_element(By.CLASS_NAME, 'weather_table').text
word

'습도 76% 체감 23.3° 동풍 2.2m/s'

In [59]:
word_list = word.split(" ")
word_list[1], word_list[3], word_list[5]

('76%', '23.3°', '2.2m/s')

In [60]:
browser.find_elements(By.CLASS_NAME, 'weather_table')

[<selenium.webdriver.remote.webelement.WebElement (session="4b4b31c465ec3843fa5a5a620e679ef9", element="d16bcdfa-6e05-4881-9b9e-e2ddffbc5962")>,
 <selenium.webdriver.remote.webelement.WebElement (session="4b4b31c465ec3843fa5a5a620e679ef9", element="669288c0-8f33-4416-99c3-1f4459e577e8")>,
 <selenium.webdriver.remote.webelement.WebElement (session="4b4b31c465ec3843fa5a5a620e679ef9", element="ec12240f-3101-4e8d-9ee8-86971681b257")>,
 <selenium.webdriver.remote.webelement.WebElement (session="4b4b31c465ec3843fa5a5a620e679ef9", element="b6d6d3e0-7db6-4a96-8182-4bd73f2757e8")>]

In [61]:
element_list = browser.find_elements(By.CLASS_NAME, 'weather_table')

for element in element_list:
    print(element.text)

습도 76% 체감 23.3° 동풍 2.2m/s
미세 좋음 초미세 좋음 일몰 19:52




In [62]:
url = "https://finance.naver.com/"
browser.get(url)

In [63]:
news = browser.find_element(By.CLASS_NAME, 'section_strategy').text

In [64]:
news_list = news.split("\n")
news_list

['주요뉴스',
 '美 긴축 장기화 우려에 일제 하락…日 0.13%↓[Asia오전]',
 '[오전시황] 코스피, 캐나다 깜짝 금리 인상 여파에 2610선 아래서 출발',
 '[굿모닝 증시]“美 기술주 중심 하락…선물옵션 동시 만기일 外人 동향 주시”',
 '주요국 긴축 경계…환율, 1310원 진입 시도[외환브리핑]',
 '쉬어가는 나스닥 1.29%↓ 지방은행 소생은 새 희망 [뉴욕마감]',
 '펜스 前 美부통령 대선출마…뉴욕증시, 고점 부담 속 혼조 [모닝브리핑]',
 '주요뉴스 더보기']

## 구글 뉴스 크롤링

In [85]:
url = "https://www.google.com/search?q=%EC%82%BC%EC%84%B1%EB%AC%BC%EC%82%B0&rlz=1C1GCEU_koKR1061KR1061&source=lnms&tbm=nws&sa=X&ved=2ahUKEwjL-tWkgLP_AhWDaN4KHaBRADQQ_AUoAXoECAEQAw&biw=1462&bih=794&dpr=1.25"
browser.get(url)
browser.title

'삼성물산 - Google 搜尋'

In [86]:
# [1] 첫번째 뉴스 제목
title = browser.find_element(By.CLASS_NAME, 'n0jPhd').text
title

"삼성물산-전력거래소, 공동주택 에너지 절감 위해 '맞손'"

In [87]:
# [2] 첫번째 뉴스 언론사
company = browser.find_element(By.CLASS_NAME, 'MgUUmf').text
company

'경기신문'

In [88]:
# [3] 첫번째 뉴스 내용
# news = browser.find_element(By.CLASS_NAME, 'WlydOe').text
news = browser.find_element(By.CLASS_NAME, 'GI74Re').text
news

'삼성물산 건설부문(이하 삼성물산)과 전력거래소는 공동주택 에너지 절감과 에너지쉼표 활성화를 위한 전략적 협업을 추진한다.'

In [89]:
# [4] 첫번째 뉴스 링크
print(browser.find_element(By.CLASS_NAME, 'WlydOe').get_attribute('href'))

https://www.kgnews.co.kr/news/article.html?no=751089


## 네이버 뉴스 크롤링

In [100]:
url = "https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid1=105&sid2=731"
browser.get(url)
browser.title

'IT/과학 홈 : 네이버 뉴스'

In [106]:
# 첫번째 
# 뉴스 제목
parent = browser.find_element(By.CLASS_NAME, 'photo')
title = parent.find_element(By.TAG_NAME, 'img').get_attribute('alt')
print(title)

‘낸드’ 반격나선 SK하이닉스…세계 최고층 238단 4D 낸드 양산


In [107]:
# 뉴스 내용
news = browser.find_element(By.CLASS_NAME, 'lede').text
print(news)

176단보다 효율 34% 높이고 데이터 전송도 1.5배 빨라 스마트폰 기업과 인증과정 기술력으로 반등기에 대비 SK하이닉스가 업 …


In [105]:
# 뉴스 링크
parent = browser.find_element(By.CLASS_NAME, 'photo')
link = parent.find_element(By.TAG_NAME, 'a').get_attribute('href')
print(link)

https://n.news.naver.com/mnews/article/009/0005141234?sid=105


In [108]:
# 뉴스 언론사
company = browser.find_element(By.CLASS_NAME, 'writing').text
print(company)

매일경제
